In [1]:
import pandas as pd
import numpy as np
import acessos as ac
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_inicio = '2023-01-01'
data_fim = '2023-09-01'

#### Trazendo dados geral

In [3]:
query_cx = f"""

select 
  dt_mes,
  hierarquia_1, 
  hierarquia_4, 
  count(distinct cpf_customer) as clientes_cx
from 
  (
    SELECT 
      date_trunc(
        'month', 
        cast(c.dt_chat as date)
      ) as dt_mes, 
      c.cpf_customer
      , 
      max(nm_tag_hierarquia_1) as hierarquia_1, 
      max(nm_tag_hierarquia_4) as hierarquia_4 
    FROM 
      cx_curated_zone.cx_contact_rate c 
      LEFT JOIN cx_curated_zone.helpshift_issues AS issues_macro ON c.id_issue = issues_macro.id_issue 
      LEFT JOIN cx_curated_zone.helpshift_tag_niveis AS helpshift_tag ON c.id_issue = helpshift_tag.id_issue 
      LEFT JOIN processed_zone_api_cxm_tags.tags_hierarquia_gerencial as novas_tags ON trim(
        lower(n3)
      ) = trim(
        lower(nm_tag_n3)
      ) 
    WHERE 
      dt_chat >= to_date('{data_inicio}', 'yyyy-mm-dd') 
      and dt_chat < to_date('{data_fim}', 'yyyy-mm-dd') 
      AND c.id_issue is not null 
      and nm_tag_hierarquia_1 <> 'Crédito e Cobrança' 
      AND issues_macro.ds_entidade_resolucao = 'agente' 
    GROUP BY 
      1, 
      2
  ) 
group by 
  1, 
  2,
  3


"""

In [4]:
df_count_cx = ac.df_athena_q('flavia-costa', query_cx)

#### Trazendo Recorte

In [5]:
query_recorte = f"""

WITH chats AS (
  SELECT 
    cpf_customer, 
    count(DISTINCT id_issue) qtd_chats 
  FROM 
    cx_curated_zone.helpshift_issues 
  WHERE 
    dt_criacao_chat >= to_date('2022-07-01', 'yyyy-mm-dd') 
    AND dt_criacao_chat < to_date('{data_fim}', 'yyyy-mm-dd') 
  GROUP BY 
    1
), 
chats_recorte_detalhado AS (
  SELECT 
    date_trunc(
      'month', 
      cast(c.dt_chat as date)
    ) as dt_mes,
    (nm_tag_hierarquia_1) as hierarquia_1, 
    (nm_tag_hierarquia_4) as hierarquia_4, 
    c.cpf_customer, 
    max(
      case when tipo_de_tag = 'Problema' then 1 else 0 end
    ) as flag_problema 
    , 
    max(nr_tpr) as nr_tpr, 
    max(nr_tma) as nr_tma, 
    max(nr_tmat) as nr_tmat
  FROM 
    cx_curated_zone.cx_contact_rate c 
    LEFT JOIN cx_curated_zone.helpshift_issues AS issues_macro ON c.id_issue = issues_macro.id_issue 
    LEFT JOIN cx_curated_zone.helpshift_tag_niveis AS helpshift_tag ON c.id_issue = helpshift_tag.id_issue 
    LEFT JOIN processed_zone_api_cxm_tags.tags_hierarquia_gerencial as novas_tags ON trim(
      lower(n3)
    ) = trim(
      lower(nm_tag_n3)
    ) 
    LEFT JOIN chats AS cht ON cht.cpf_customer = c.cpf_customer 
  WHERE 
    dt_chat >= to_date('{data_inicio}', 'yyyy-mm-dd') 
    and dt_chat < to_date('{data_fim}', 'yyyy-mm-dd') 
    AND c.id_issue is not null 
    and nm_tag_hierarquia_1 <> 'Crédito e Cobrança' 
    AND issues_macro.ds_entidade_resolucao = 'agente' --RECORTE
    and cht.cpf_customer is not NULL 
    AND cht.qtd_chats < 4 
    and issues_macro.nr_tmat > 90 
  GROUP BY 
    1, 
    2,
    3,
    4
), 
de_para AS (
  SELECT 
    DISTINCT dt_mes, 
    date_add('day', -91, dt_mes) as periodo_antes_inicio, 
    date_add('day', -1, dt_mes) as periodo_antes_fim, 
    date_add('month', 1, dt_mes) as periodo_depois_inicio, 
    date_add('month', 3, dt_mes) as periodo_depois_fim, 
    date_add('month', 2, dt_mes) as periodo_depois30_fim 
  FROM 
    chats_recorte_detalhado
), 
spending_antes_total_ AS (
  SELECT 
    distinct s.cpf, 
    s.id_autorizacao, 
    c.dt_mes, 
    periodo_antes_inicio, 
    periodo_antes_fim, 
    periodo_depois_inicio, 
    periodo_depois_fim, 
    periodo_depois30_fim, 
    s.vl_transacao as vl_compra_antes_90d --, count(distinct s.id_autorizacao) as qt_compra_antes_90d
    , 
    bc.dt_criacao_conta as dt_criacao_conta, 
    s.dt_dia as dt_last_use_credit, 
    a.dt_first_use_credit as dt_first_use_credit, 
    gc.vl_current_limit as vl_current_limit 
  FROM 
    platform_curated_zone.transaction_dedicada_spending s 
    INNER JOIN de_para c ON (
      cast(s.dt_dia as date) >= c.periodo_antes_inicio 
      AND cast(s.dt_dia as date) <= c.periodo_antes_fim
    ) --IGUAL OU MAIOR E IGUAL
    LEFT JOIN (
      SELECT 
        cpf, 
        id_customer, 
        min(cd_yearmonth) as dt_criacao_conta 
      FROM 
        customer_curated_zone.ca_book_cliente 
      GROUP BY 
        1, 
        2
    ) bc ON (bc.cpf = s.cpf) 
    LEFT JOIN customer_curated_zone.ca_book_status_cliente a ON (a.id_customer = bc.id_customer) --limite
    LEFT JOIN customer_curated_zone.ca_book_gestao_carteira gc ON (
      s.cpf = gc.cpf 
      AND cast(
        replace(
          substring(
            cast(c.dt_mes as varchar), 
            1, 
            7
          ), 
          '-', 
          ''
        ) as double
      ) = gc.cd_yearmonth
    ) 
  WHERE 
    s.vl_transacao > 0 
    AND bc.dt_criacao_conta IS NOT NULL 
    AND gc.vl_current_limit IS NOT NULL
), 
spending_antes_total AS (
  select 
    cpf, 
    dt_mes, 
    periodo_antes_inicio, 
    periodo_antes_fim, 
    periodo_depois_fim, 
    periodo_depois30_fim, 
    periodo_depois_inicio, 
    dt_criacao_conta, 
    sum(vl_compra_antes_90d) as vl_compra_antes_90d, 
    count(distinct id_autorizacao) as qt_compra_antes_90d, 
    max(dt_last_use_credit) as dt_last_use_credit, 
    min(dt_first_use_credit) as dt_first_use_credit, 
    max(vl_current_limit) as vl_current_limit 
  from 
    spending_antes_total_ 
  group by 
    1, 
    2, 
    3, 
    4, 
    5, 
    6, 
    7, 
    8
), 
base_total_add_chats AS (
  SELECT 
    s.cpf, 
    s.dt_mes as dt_mes_true, 
    s.periodo_antes_inicio, 
    s.periodo_antes_fim, 
    s.periodo_depois_inicio, 
    s.periodo_depois_fim, 
    s.periodo_depois30_fim, 
    s.vl_compra_antes_90d, 
    s.qt_compra_antes_90d, 
    s.dt_last_use_credit, 
    s.dt_first_use_credit, 
    s.dt_criacao_conta, 
    s.vl_current_limit, 
    c.*, 
    case when c.cpf_customer is null then 0 else 1 end as usou_chat 
  FROM 
    chats_recorte_detalhado c 
    LEFT JOIN spending_antes_total s on (
      s.cpf = c.cpf_customer 
      and c.dt_mes = s.dt_mes
    )
), 
spending_depois_30 as (
  select 
    c.cpf, 
    c.dt_mes_true, 
    sum(s_30.vl_transacao) as vl_compra_depois_30d, 
    count(distinct s_30.id_autorizacao) as qt_compras_depois_30d 
  from 
    base_total_add_chats c 
    left join platform_curated_zone.transaction_dedicada_spending s_30 on (
      c.cpf = s_30.cpf 
      and cast(s_30.dt_dia as date) >= c.periodo_depois_inicio 
      and cast(s_30.dt_dia as date) <= c.periodo_depois30_fim
    ) 
  group by 
    1, 
    2
), 
spending_depois_60 as (
  select 
    c.cpf, 
    c.dt_mes_true, 
    sum(s_60.vl_transacao) as vl_compra_depois_60d, 
    count(distinct s_60.id_autorizacao) as qt_compras_depois_60d 
  from 
    base_total_add_chats c 
    left join platform_curated_zone.transaction_dedicada_spending s_60 on (
      c.cpf = s_60.cpf 
      and cast(s_60.dt_dia as date) >= c.periodo_depois_inicio 
      and cast(s_60.dt_dia as date) <= c.periodo_depois_fim
    ) 
  group by 
    1, 
    2
), 
union_ltv as (
  select 
    distinct mesref, 
    tipo, 
    dias_atraso, 
    cpf, 
    spending_credito, 
    rec_itr, 
    rec_recarga, 
    rec_demais_encargos, 
    desp_pdd, 
    rec_juros, 
    custos, 
    ltv 
  from 
    public.rentabilidade_cartoes_diego_camilo
) 
SELECT 
  a.cpf, 
  a.dt_mes_true, 
  a.periodo_antes_inicio, 
  a.periodo_antes_fim, 
  a.periodo_depois_inicio, 
  a.periodo_depois_fim, 
  a.periodo_depois30_fim, 
  a.qt_compra_antes_90d, 
  a.dt_last_use_credit, 
  a.vl_current_limit, 
  a.flag_problema, 
  a.nr_tpr, 
  a.nr_tma, 
  a.nr_tmat, 
  a.usou_chat, 
  a.hierarquia_1, 
  a.hierarquia_4, 
  max(a.vl_compra_antes_90d) vl_compra_antes_90d, 
  min(a.dt_first_use_credit) dt_first_use_credit, 
  max(gro_.vl_renda_declarada) vl_renda_declarada, 
  min(a.dt_criacao_conta) dt_criacao_conta,
  min(gro_.dt_opening_account) as dt_opening_account,
  count(
    distinct substring(
      cast(dt_event as varchar), 
      1, 
      10
    )
  ) as dias_uso_app, 
  max(d1.vl_compra_depois_30d) vl_compra_depois_30d, 
  max(d1.qt_compras_depois_30d) qt_compras_depois_30d, 
  max(d2.vl_compra_depois_60d) vl_compra_depois_60d, 
  max(d2.qt_compras_depois_60d) qt_compras_depois_60d, 
  max(l.spending_credito) as spending_ltv, 
  max(l.rec_itr) as rec_itr, 
  max(l.rec_recarga) as rec_recarga, 
  max(l.rec_demais_encargos) as rec_demais_encargos, 
  max(l.desp_pdd) as desp_pdd, 
  max(l.rec_juros) as rec_juros, 
  max(l.custos) as custos, 
  max(l.ltv) as ltv, 
  max(l.dias_atraso) as dias_atraso_ltv 
FROM 
  base_total_add_chats a 
  LEFT JOIN customer_curated_zone.ca_book_cliente b_c ON (a.cpf = b_c.cpf) 
  LEFT JOIN customer_curated_zone.ca_book_growth gro_ ON gro_.id_customer = b_c.id_customer 
  LEFT JOIN customer_curated_zone.ca_analitico_geoloc ap on (
    ap.cpf = a.cpf 
    and ap.dt_event >= a.periodo_antes_inicio 
    and ap.dt_event <= a.periodo_antes_fim
  ) 
  LEFT JOIN spending_depois_30 d1 on (
    a.cpf = d1.cpf 
    and a.dt_mes_true = d1.dt_mes_true
  ) 
  LEFT JOIN spending_depois_60 d2 on (
    a.cpf = d2.cpf 
    and a.dt_mes_true = d2.dt_mes_true
  ) 
  left join union_ltv l on (
    a.cpf = l.cpf 
    and a.dt_mes_true = cast(
      concat(l.mesref, '-01') as date
    )
  ) 
where 
  a.usou_chat = 1 
group by 
  1, 
  2, 
  3, 
  4, 
  5, 
  6, 
  7, 
  8, 
  9, 
  10, 
  11, 
  12, 
  13, 
  14, 
  15, 
  16, 
  17

"""

In [6]:
df = ac.df_athena_q('flavia-costa', query_recorte)

In [7]:
len(df)

678285

In [8]:
missing_zero = ['qt_compra_antes_90d', 'vl_compra_antes_90d', 'vl_compra_depois_60d', 'spending_ltv', 'ltv', 'dias_atraso_ltv', 'rec_itr',
					'rec_recarga', 'rec_demais_encargos', 'desp_pdd', 'rec_juros', 'custos']

for column in missing_zero:
    df[column].fillna(0, inplace = True)

Criação de variáveis

In [9]:
df['dt_opening_account'] = pd.to_datetime(df['dt_opening_account'], format='%Y-%m-%d')
df['dt_mes_true'] = pd.to_datetime(df['dt_mes_true'], format='%Y-%m-%d')
df['cc_time'] = (df['dt_mes_true'] - df['dt_opening_account'])/ np.timedelta64(1, 'D')
    
df['dt_first_use_credit'] = pd.to_datetime(df['dt_first_use_credit'], format='%Y-%m-%d')
df['dias_ativacao'] = (df['dt_mes_true'] - df['dt_first_use_credit'])/ np.timedelta64(1, 'D')
       
df['dt_last_use_credit'] = pd.to_datetime(df['dt_last_use_credit'], format='%Y-%m-%d', errors = 'coerce')
df['dias_desde_ultima_compra'] = (df['dt_mes_true'] - df['dt_last_use_credit']).dt.days
   
df['vl_compra_antes_90d_log'] = np.log(df.vl_compra_antes_90d) #Ajuste da variável


/home/flavia.costa/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Lidando com missings

In [10]:
for column in df.columns:
    if df[column].dtype == 'object':
        df[column].fillna(df[column].mode()[0], inplace = True)
    elif column == 'vl_compra_antes_90d_log':
        df[column].fillna(0, inplace = True)
    elif df[column].dtype != 'object':
        df[column].fillna(df[column].median(), inplace = True)

Função de normalização

In [11]:
def normalizando(x, var_min, var_max):
    y = (x - var_min) / (var_max - var_min)
    return y

In [12]:
df_n = pd.DataFrame()

df_n['cc_time'] = normalizando(df['cc_time'], 0.19, 2062.09)
df_n['vl_current_limit'] = normalizando(df['vl_current_limit'], 0, 12300)
df_n['dias_uso_app'] = normalizando(df['dias_uso_app'], 0, 35)
df_n['vl_renda_declarada'] = normalizando(df['vl_renda_declarada'], 0, 999999.99)
df_n['dias_ativacao'] = normalizando(df['dias_ativacao'], 0, 1812.25)
df_n['qt_compra_antes_90d'] = normalizando(df['qt_compra_antes_90d'], 1, 78)
df_n['dias_desde_ultima_compra'] = normalizando(df['dias_desde_ultima_compra'], 1, 77)


## Base ano anterior

In [13]:
query_aa = f"""

WITH chats AS (
  SELECT 
    cpf_customer, 
    count(DISTINCT id_issue) qtd_chats 
  FROM 
    cx_curated_zone.helpshift_issues 
  WHERE 
    dt_criacao_chat >= to_date('2021-01-01', 'yyyy-mm-dd') 
    AND dt_criacao_chat < to_date('{data_inicio}', 'yyyy-mm-dd') 
  GROUP BY 
    1
), 
chats_agg AS (
  SELECT 
    date_trunc(
      'month', 
      cast(c.dt_chat as date)
    ) as dt_mes, 
    c.cpf_customer -- problema?
    , 
    max(
      case when tipo_de_tag = 'Problema' then 1 else 0 end
    ) as flag_problema --finalizado por automacao
    , 
    max(nr_tpr) as nr_tpr, 
    max(nr_tma) as nr_tma, 
    max(nr_tmat) as nr_tmat, 
    max(nm_tag_hierarquia_1) as hierarquia_1 
  FROM 
    cx_curated_zone.cx_contact_rate c 
    LEFT JOIN cx_curated_zone.helpshift_issues AS issues_macro ON c.id_issue = issues_macro.id_issue 
    LEFT JOIN cx_curated_zone.helpshift_tag_niveis AS helpshift_tag ON c.id_issue = helpshift_tag.id_issue 
    LEFT JOIN processed_zone_api_cxm_tags.tags_hierarquia_gerencial as novas_tags ON trim(
      lower(n3)
    ) = trim(
      lower(nm_tag_n3)
    ) 
    INNER JOIN chats AS cht ON cht.cpf_customer = c.cpf_customer 
    AND cht.qtd_chats < 4 
  WHERE 
    dt_chat >= to_date('2022-01-01', 'yyyy-mm-dd') 
    and dt_chat < to_date('{data_inicio}', 'yyyy-mm-dd') 
    AND c.id_issue is not null 
    and issues_macro.nr_tmat > 90 
    and nm_tag_hierarquia_1 <> 'Crédito e Cobrança' 
  GROUP BY 
    1, 
    2
), 
de_para AS (
  SELECT 
    DISTINCT dt_mes, 
    date_add('day', -91, dt_mes) as periodo_antes_inicio, 
    date_add('day', -1, dt_mes) as periodo_antes_fim, 
    date_add('month', 9, dt_mes) as periodo_depois_inicio, 
    date_add('month', 12, dt_mes) as periodo_depois_fim 
  FROM 
    chats_agg
), 
spending_antes_total_ AS (
  SELECT 
    distinct s.cpf, 
    s.id_autorizacao, 
    c.dt_mes, 
    periodo_antes_inicio, 
    periodo_antes_fim, 
    periodo_depois_inicio, 
    periodo_depois_fim, 
    s.vl_transacao as vl_compra_antes_90d --, count(distinct s.id_autorizacao) as qt_compra_antes_90d
    , 
    bc.dt_criacao_conta as dt_criacao_conta, 
    s.dt_dia as dt_last_use_credit, 
    a.dt_first_use_credit as dt_first_use_credit, 
    gc.vl_current_limit as vl_current_limit 
  FROM 
    platform_curated_zone.transaction_dedicada_spending s 
    INNER JOIN de_para c ON (
      cast(s.dt_dia as date) >= c.periodo_antes_inicio 
      AND cast(s.dt_dia as date) <= c.periodo_antes_fim
    ) --IGUAL OU MAIOR E IGUAL
    LEFT JOIN (
      SELECT 
        cpf, 
        id_customer, 
        min(cd_yearmonth) as dt_criacao_conta 
      FROM 
        customer_curated_zone.ca_book_cliente 
      GROUP BY 
        1, 
        2
    ) bc ON (bc.cpf = s.cpf) 
    LEFT JOIN customer_curated_zone.ca_book_status_cliente a ON (a.id_customer = bc.id_customer) --limite
    LEFT JOIN customer_curated_zone.ca_book_gestao_carteira gc ON (
      s.cpf = gc.cpf 
      AND cast(
        replace(
          substring(
            cast(c.dt_mes as varchar), 
            1, 
            7
          ), 
          '-', 
          ''
        ) as double
      ) = gc.cd_yearmonth
    ) 
  WHERE 
    s.vl_transacao > 0 
    AND bc.dt_criacao_conta IS NOT NULL 
    AND gc.vl_current_limit IS NOT NULL
), 
spending_antes_total AS (
  select 
    cpf, 
    dt_mes, 
    periodo_antes_inicio, 
    periodo_antes_fim, 
    periodo_depois_fim, 
    periodo_depois_inicio, 
    dt_criacao_conta, 
    sum(vl_compra_antes_90d) as vl_compra_antes_90d, 
    count(distinct id_autorizacao) as qt_compra_antes_90d, 
    max(dt_last_use_credit) as dt_last_use_credit, 
    min(dt_first_use_credit) as dt_first_use_credit, 
    max(vl_current_limit) as vl_current_limit 
  from 
    spending_antes_total_ 
  group by 
    1, 
    2, 
    3, 
    4, 
    5, 
    6, 
    7
), 
base_total_add_chats AS (
  SELECT 
    s.cpf, 
    s.dt_mes as dt_mes_true, 
    s.periodo_antes_inicio, 
    s.periodo_antes_fim, 
    s.periodo_depois_inicio, 
    s.periodo_depois_fim, 
    s.vl_compra_antes_90d, 
    s.qt_compra_antes_90d, 
    s.dt_last_use_credit, 
    s.dt_first_use_credit, 
    s.dt_criacao_conta, 
    s.vl_current_limit, 
    c.*, 
    case when c.cpf_customer is null then 0 else 1 end as usou_chat 
  FROM 
    spending_antes_total s 
    LEFT JOIN chats_agg c on (
      s.cpf = c.cpf_customer 
      and c.dt_mes = s.dt_mes
    )
), 
spending_depois as (
  select 
    c.cpf, 
    c.dt_mes_true, 
    sum(s_60.vl_transacao) as vl_compra_depois_ano, 
    count(distinct s_60.id_autorizacao) as qt_compras_depois_ano 
  from 
    base_total_add_chats c 
    left join platform_curated_zone.transaction_dedicada_spending s_60 on (
      c.cpf = s_60.cpf 
      and cast(s_60.dt_dia as date) >= c.periodo_depois_inicio 
      and cast(s_60.dt_dia as date) <= c.periodo_depois_fim
    ) 
  group by 
    1, 
    2
), 
base_completa as (
  SELECT 
    a.cpf, 
    a.dt_mes_true, 
    a.qt_compra_antes_90d, 
    a.dt_last_use_credit, 
    a.vl_current_limit, 
    a.flag_problema, 
    a.nr_tpr, 
    a.nr_tma, 
    a.nr_tmat, 
    a.usou_chat, 
    a.hierarquia_1, 
    max(a.vl_compra_antes_90d) vl_compra_antes_90d, 
    min(a.dt_first_use_credit) dt_first_use_credit, 
    min(a.dt_criacao_conta) dt_criacao_conta, 
    count(
      distinct substring(
        cast(dt_event as varchar), 
        1, 
        10
      )
    ) as dias_uso_app, 
    max(d2.vl_compra_depois_ano) vl_compra_depois_ano, 
    max(d2.qt_compras_depois_ano) qt_compras_depois_ano 
  FROM 
    base_total_add_chats a 
    LEFT JOIN customer_curated_zone.ca_book_cliente b_c ON (a.cpf = b_c.cpf) 
    LEFT JOIN customer_curated_zone.ca_analitico_geoloc ap on (
      ap.cpf = a.cpf 
      and ap.dt_event >= a.periodo_antes_inicio 
      and ap.dt_event <= a.periodo_antes_fim
    ) 
    LEFT JOIN spending_depois d2 on (
      a.cpf = d2.cpf 
      and a.dt_mes_true = d2.dt_mes_true
    ) 
  where 
    a.usou_chat = 1 
  group by 
    1, 
    2, 
    3, 
    4, 
    5, 
    6, 
    7, 
    8, 
    9, 
    10, 
    11
) 
select 
  * 
from 
  base_completa



"""

## Calculando a taxa de churn por grupo

In [14]:
df_aa = ac.df_athena_q('flavia-costa', query_aa)

missing_zero = ['qt_compra_antes_90d', 'vl_compra_antes_90d', 'vl_compra_depois_ano']

for column in missing_zero:
    df_aa[column].fillna(0, inplace = True)

df_aa['dt_mes_true'] = pd.to_datetime(df_aa['dt_mes_true'])
df_aa['dt_last_use_credit'] = pd.to_datetime(df_aa['dt_last_use_credit'])
df_aa['dt_criacao_conta_str'] = df_aa['dt_criacao_conta'].astype('string') + '01'
df_aa['dt_criacao_conta_date'] = pd.to_datetime(df_aa['dt_criacao_conta_str'], format='%Y%m%d', errors='ignore')

# variaveis dias
df_aa['dias_desde_ultima_compra'] = (df_aa['dt_mes_true'] - df_aa['dt_last_use_credit']).dt.days 
df_aa['cc_time'] = (df_aa['dt_mes_true'] - df_aa['dt_criacao_conta_date']).dt.days 
df_aa['dias_ativacao'] = (df_aa['dt_mes_true'] - df_aa['dt_first_use_credit']).dt.days 

vars_rfm = ['dias_ativacao', 'cc_time', 'dias_desde_ultima_compra', 'qt_compra_antes_90d', 'vl_compra_antes_90d']

for column in vars_rfm:
  if df_aa[column].dtype == 'float' or df_aa[column].dtype == 'int':
    nm_col = column + "_fx"
    df_aa[nm_col] = pd.qcut(df_aa[column].rank(method='first'), q=3, precision=0, labels = ['A', 'B', 'C'])

df_aa['segmento'] = df_aa.cc_time_fx.astype("string") + df_aa.dias_desde_ultima_compra_fx.astype("string") + df_aa.qt_compra_antes_90d_fx.astype("string") + df_aa.vl_compra_antes_90d_fx.astype("string")
df_aa['churn_depois'] = np.where(df_aa.vl_compra_depois_ano > 0, 0, 1)
tabela_churn = df_aa[['churn_depois', 'segmento']].groupby('segmento').mean()
tabela_churn = tabela_churn.reset_index()
tabela_churn['LT'] = (1 / (tabela_churn['churn_depois']/12))

## Calcula Life time mediano dos clientes do recorte

In [15]:
tabela_churn['LT'].min()

17.640552995391705

In [16]:
tabela_churn['LT'].median()

34.07067961165048

In [17]:
tabela_churn['LT'].max()

64.99433198380567

In [18]:
lt_mediano= tabela_churn['LT'].median()

In [19]:
#MARCANDO O CHURN NA BASE ATUAL
for column in vars_rfm:
    if df[column].dtype == 'float' or df[column].dtype == 'int':
      nm_col = column + "_fx"
      df[nm_col] = pd.qcut(df[column].rank(method='first'), q=3, precision=0, labels = ['A', 'B', 'C'])

df['segmento'] = df.cc_time_fx.astype("string") + df.dias_desde_ultima_compra_fx.astype("string") + df.qt_compra_antes_90d_fx.astype("string") + df.vl_compra_antes_90d_fx.astype("string")
df = df.join(tabela_churn.set_index('segmento'), on = 'segmento', how = 'left' )

In [20]:
df.head()

,cpf,dt_mes_true,periodo_antes_inicio,periodo_antes_fim,periodo_depois_inicio,periodo_depois_fim,periodo_depois30_fim,qt_compra_antes_90d,dt_last_use_credit,vl_current_limit,...,dias_desde_ultima_compra,vl_compra_antes_90d_log,dias_ativacao_fx,cc_time_fx,dias_desde_ultima_compra_fx,qt_compra_antes_90d_fx,vl_compra_antes_90d_fx,segmento,churn_depois,LT
0,04135928390,2023-05-01,2023-01-30,2023-04-30,2023-06-01,2023-08-01,2023-07-01,1.0,2023-04-25,200.0,...,6.0,3.514526,A,A,A,B,B,AABB,0.301348,39.821038
1,13238377442,2023-04-01,2022-12-31,2023-03-31,2023-05-01,2023-07-01,2023-06-01,8.0,2023-03-27,600.0,...,5.0,6.766192,C,A,A,B,B,AABB,0.301348,39.821038
2,05917322708,2023-08-01,2023-05-02,2023-07-31,2023-09-01,2023-11-01,2023-10-01,32.0,2023-07-02,3200.0,...,30.0,8.214232,A,C,C,C,C,CCCC,0.482648,24.862837
3,12671934416,2023-04-01,2022-12-31,2023-03-31,2023-05-01,2023-07-01,2023-06-01,15.0,2023-02-25,660.0,...,35.0,7.181850,C,C,C,C,C,CCCC,0.482648,24.862837
4,07490148308,2023-05-01,2023-01-30,2023-04-30,2023-06-01,2023-08-01,2023-07-01,14.0,2023-04-28,15000.0,...,3.0,8.323128,C,A,A,C,C,AACC,0.329471,36.422053


In [21]:
df['LT'].median()

32.03055229142186

## Parametros modelo churn

In [22]:
#Parametros modelo - regressão logística do estudo prévio
c_intercepto = -0.1635
c_usou_chat = -0.0655
c_flag_problema = -0.0993
c_vl_compra_antes_90d_log = 0.1318
c_cc_time = -0.6206
c_vl_current_limit = 0.3866
c_dias_uso_app = 0.2392
c_vl_renda_declarada = -5.3729
c_dias_ativacao = 0.3947
c_qt_compra_antes_90d = 0.2761
c_dias_desde_ultima_compra = -2.9933

queda_cr = 0.1

Loop simulações

In [23]:
#iteracoes = 500

In [24]:
#df_simulacoes = pd.DataFrame()

## Inputando score de churn pra todos

In [25]:
df['usou_chat_sim'] = 0
df['flag_problema_sim'] = 0

df['rawPrediction'] = c_intercepto  + ((df.usou_chat_sim * c_usou_chat) + (df.flag_problema_sim * c_flag_problema) 
  + (df.vl_compra_antes_90d_log * c_vl_compra_antes_90d_log) 
  + (df_n.cc_time * c_cc_time)
  + (df_n.vl_current_limit * c_vl_current_limit) 
  + (df_n.dias_uso_app * c_dias_uso_app) 
  + (df_n.vl_renda_declarada * c_vl_renda_declarada) 
  + (df_n.dias_ativacao * c_dias_ativacao) 
  + (df_n.qt_compra_antes_90d * c_qt_compra_antes_90d)
  + (df_n.dias_desde_ultima_compra * c_dias_desde_ultima_compra))

df['score_compra_depois']  =  1/(1 + np.exp(-1 * df.rawPrediction))
df['score_compra_depois'] = np.where(df['score_compra_depois'] < 0, 0, df['score_compra_depois'])
df['score_compra_depois'] = np.where(df['score_compra_depois'] > 1, 1, df['score_compra_depois'])

In [26]:
df['score_compra_depois'].median()

0.4433404570008307

## Levantamento de clientes no recorte que entraram em churn (não compraram após 2 meses)

In [27]:
df['teve_compra_depois'] = np.where(df.vl_compra_depois_60d > 0 , 1, 0)
df['teve_compra_depois'].value_counts()

1    339483
0    338802
Name: teve_compra_depois, dtype: int64

In [28]:
df_agg_recorte = df.groupby(['dt_mes_true', 'hierarquia_1', 'hierarquia_4']).agg({'cpf':['nunique']})
df_agg_recorte.columns = df_agg_recorte.columns.droplevel(0)
df_agg_recorte = df_agg_recorte.reset_index()
df_agg_recorte.columns = ['dt_mes', 'hierarquia_1', 'hierarquia_4', 'clientes_recorte_total']
df_agg_recorte.head()

,dt_mes,hierarquia_1,hierarquia_4,clientes_recorte_total
0,2023-01-01,Assunto não identificado,Assunto não identificado,521
1,2023-01-01,Atendimento,Atendimento de terceiros,193
2,2023-01-01,Atendimento,Canais de atendimento,59
3,2023-01-01,Atendimento,Chat duplicado,11
4,2023-01-01,Atendimento,Chat sem retorno,852


In [29]:
df_agg_churn = df.query('teve_compra_depois == 0').groupby(['dt_mes_true', 'hierarquia_1', 'hierarquia_4']).agg({'cpf':['nunique']})
df_agg_churn.columns = df_agg_churn.columns.droplevel(0)
df_agg_churn = df_agg_churn.reset_index()
df_agg_churn.columns = ['dt_mes', 'hierarquia_1', 'hierarquia_4', 'clientes_recorte_churn']
df_agg_churn.head()


,dt_mes,hierarquia_1,hierarquia_4,clientes_recorte_churn
0,2023-01-01,Assunto não identificado,Assunto não identificado,72
1,2023-01-01,Atendimento,Atendimento de terceiros,22
2,2023-01-01,Atendimento,Canais de atendimento,11
3,2023-01-01,Atendimento,Chat duplicado,1
4,2023-01-01,Atendimento,Chat sem retorno,127


In [30]:
df_count_cx.head()

,dt_mes,hierarquia_1,hierarquia_4,clientes_cx
0,2023-06-01,Cartão,Dúvidas sobre pagamento de fatura,2467
1,2023-08-01,Cartão,Problemas com a compra,7393
2,2023-05-01,Tecnologia,Recebimento de PIX,5356
3,2023-05-01,Conta,Recebimento de PIX,7604
4,2023-08-01,Crédito,Dúvidas sobre limite de crédito,9678


### Definindo qtos clientes já entrariam em churn independente do atendimento

In [31]:
df['fx_score'] = np.round(df['score_compra_depois'],1)
  
#tabela KS
agg_df_compra_sim = df[df['teve_compra_depois'] == 1][['fx_score', 'cpf']].groupby('fx_score').count()
agg_df_compra_nao = df[df['teve_compra_depois'] == 0][['fx_score', 'cpf']].groupby('fx_score').count()
agg_df_compra = agg_df_compra_sim.join(agg_df_compra_nao, lsuffix='nao', rsuffix='sim')
agg_df_compra = agg_df_compra.reset_index()
agg_df_compra['cumperc_nao'] = agg_df_compra['cpfnao'].cumsum()/agg_df_compra['cpfnao'].sum()*100
agg_df_compra['cumperc_sim'] = agg_df_compra['cpfsim'].cumsum()/agg_df_compra['cpfsim'].sum()*100
agg_df_compra['dif'] = agg_df_compra['cumperc_nao'] - agg_df_compra['cumperc_sim']
agg_df_compra['dif_abs'] = np.abs(agg_df_compra['dif']) 

#quando a diferença nas distribuições ficar acima de 10%
dif_maxima = agg_df_compra['dif_abs'].max()
threshould_conservador = agg_df_compra[agg_df_compra.dif_abs == dif_maxima]['fx_score'].min()

df['compra_depois_propensao']  = np.where(df['fx_score'] > threshould_conservador, 1,0)

#inputando spending para o cliente que teve churn de fato, que foi sorteado para não ter atendimento na simulação e com baixo score de compra depois
df['flag_atribui_spending'] = np.where((df['compra_depois_propensao'] == 1) & (df['teve_compra_depois'] == 0) & (df.usou_chat_sim == 0),1,0)

In [32]:
df['flag_atribui_spending'].value_counts()

0    613487
1     64798
Name: flag_atribui_spending, dtype: int64

In [33]:
df_agg_churn_recuperados = df.query('flag_atribui_spending == 1').groupby(['dt_mes_true', 'hierarquia_1', 'hierarquia_4']).agg({'cpf':['nunique']})
df_agg_churn_recuperados.columns = df_agg_churn_recuperados.columns.droplevel(0)
df_agg_churn_recuperados = df_agg_churn_recuperados.reset_index()
df_agg_churn_recuperados.columns = ['dt_mes', 'hierarquia_1', 'hierarquia_4', 'clientes_recorte_churn_recuperados']
df_agg_churn_recuperados.head()

,dt_mes,hierarquia_1,hierarquia_4,clientes_recorte_churn_recuperados
0,2023-01-01,Assunto não identificado,Assunto não identificado,56
1,2023-01-01,Atendimento,Atendimento de terceiros,18
2,2023-01-01,Atendimento,Canais de atendimento,10
3,2023-01-01,Atendimento,Chat sem retorno,106
4,2023-01-01,Cartão,Alteração de Cadastro,283


### Estimando o gasto que os recuperados teriam nos 60 dias com base no perfil

In [34]:
df_comprou = df.query('teve_compra_depois == 1')
df_nao_comprou = df.query('flag_atribui_spending == 1')

In [35]:
for column in df_nao_comprou.columns:
    if df_nao_comprou[column].dtype == 'object':
        df_nao_comprou[column].fillna(df_nao_comprou[column].mode()[0], inplace = True)
    elif df_nao_comprou[column].dtype == 'string':
        pass
    elif df_nao_comprou[column].dtype == 'category':
        pass
    elif df_nao_comprou[column].dtype == 'datetime64[ns]':
        pass
    elif df_nao_comprou[column].dtype != 'object':
        df_nao_comprou[column].fillna(df_nao_comprou[column].median(), inplace = True)

/tmp/ipykernel_498077/710299964.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou[column].fillna(df_nao_comprou[column].mode()[0], inplace = True)
/tmp/ipykernel_498077/710299964.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou[column].fillna(df_nao_comprou[column].mode()[0], inplace = True)
/tmp/ipykernel_498077/710299964.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou[column].fill

#### Modelo linear

In [36]:
formula_text = ("""
      vl_compra_depois_60d ~ 
      C(periodo_depois30_fim) + 
        qt_compra_antes_90d +  vl_current_limit + 
        cc_time + dias_ativacao + dias_desde_ultima_compra +
       vl_compra_antes_90d + vl_renda_declarada  + dias_uso_app
      """)

model = ols(formula = formula_text, data = df_comprou).fit()

In [37]:
prediction = model.predict(df_nao_comprou)
df_nao_comprou['valor incremental bruto'] = prediction

/tmp/ipykernel_498077/829968082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou['valor incremental bruto'] = prediction


In [38]:
df_nao_comprou['valor incremental bruto'].median()

1223.0714238846524

#### Ajustando o valor incremental bruto para o caso de ser maior que o limite do cliente

In [39]:
df_nao_comprou['valor incremental bruto'] = np.where(df_nao_comprou['valor incremental bruto'] >  df_nao_comprou['vl_current_limit'], df_nao_comprou['vl_current_limit'], df_nao_comprou['valor incremental bruto'])

/tmp/ipykernel_498077/4211581140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou['valor incremental bruto'] = np.where(df_nao_comprou['valor incremental bruto'] >  df_nao_comprou['vl_current_limit'], df_nao_comprou['vl_current_limit'], df_nao_comprou['valor incremental bruto'])


### Estimando LTV

In [40]:
df_nao_comprou['LTV Receita'] = np.where(df_nao_comprou['ltv'] < 0, 0, df_nao_comprou['ltv'] * df_nao_comprou['LT'])

/tmp/ipykernel_498077/1633370652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou['LTV Receita'] = np.where(df_nao_comprou['ltv'] < 0, 0, df_nao_comprou['ltv'] * df_nao_comprou['LT'])


In [41]:
df_agg_vl_bruto_recuperados = df_nao_comprou.groupby(['dt_mes_true', 'hierarquia_1', 'hierarquia_4']).agg({'valor incremental bruto':['sum'], 'LTV Receita':['sum'], 'LT':['median']})
df_agg_vl_bruto_recuperados.columns = df_agg_vl_bruto_recuperados.columns.droplevel(0)
df_agg_vl_bruto_recuperados = df_agg_vl_bruto_recuperados.reset_index()
df_agg_vl_bruto_recuperados.columns = ['dt_mes', 'hierarquia_1', 'hierarquia_4', 'vl_bruto_recuperados_60d', 'LTV_Receita_Recuperado', 'LT_mediano_recuperados']
df_agg_vl_bruto_recuperados.head()

,dt_mes,hierarquia_1,hierarquia_4,vl_bruto_recuperados_60d,LTV_Receita_Recuperado,LT_mediano_recuperados
0,2023-01-01,Assunto não identificado,Assunto não identificado,62098.780200,70802.695710,25.841340
1,2023-01-01,Atendimento,Atendimento de terceiros,29270.059719,85805.115735,26.819843
2,2023-01-01,Atendimento,Canais de atendimento,10206.941777,4238.746010,26.248350
3,2023-01-01,Atendimento,Chat sem retorno,236645.738123,500816.391035,26.819843
4,2023-01-01,Cartão,Alteração de Cadastro,415833.840712,659358.835564,26.819843


### LTV Total para relativizar

### Unindo bases

In [42]:
df_agg_vl_bruto_recuperados['dt_mes'] = df_agg_vl_bruto_recuperados['dt_mes'].astype('object')
df_agg_churn_recuperados['dt_mes'] = df_agg_churn_recuperados['dt_mes'].astype('object')
df_agg_churn['dt_mes'] = df_agg_churn['dt_mes'].astype('object')


In [43]:
df_count_cx['dt_mes'] = df_count_cx['dt_mes'].astype('object')

In [44]:
df_agg_recorte['dt_mes'] = df_agg_recorte['dt_mes'].astype('object')

In [45]:
df_count_cx['chave'] = df_count_cx.dt_mes.astype('string') + df_count_cx.hierarquia_1.astype('string') + df_count_cx.hierarquia_4.astype('string')
df_agg_vl_bruto_recuperados['chave'] = df_agg_vl_bruto_recuperados.dt_mes.astype('string') + df_agg_vl_bruto_recuperados.hierarquia_1.astype('string') + df_agg_vl_bruto_recuperados.hierarquia_4.astype('string')
df_agg_churn_recuperados['chave'] = df_agg_churn_recuperados.dt_mes.astype('string') + df_agg_churn_recuperados.hierarquia_1.astype('string') + df_agg_churn_recuperados.hierarquia_4.astype('string')
df_agg_churn['chave'] = df_agg_churn.dt_mes.astype('string') + df_agg_churn.hierarquia_1.astype('string') + df_agg_churn.hierarquia_4.astype('string')
df_agg_recorte['chave'] = df_agg_recorte.dt_mes.astype('string') + df_agg_recorte.hierarquia_1.astype('string') + df_agg_recorte.hierarquia_4.astype('string')


In [46]:
df_count_cx['chave'] = df_count_cx['chave'].str.replace(' 00:00:00', '')
df_agg_vl_bruto_recuperados['chave'] = df_agg_vl_bruto_recuperados['chave'].str.replace(' 00:00:00', '')
df_agg_churn_recuperados['chave'] = df_agg_churn_recuperados['chave'].str.replace(' 00:00:00', '')
df_agg_churn['chave'] = df_agg_churn['chave'].str.replace(' 00:00:00', '')
df_agg_recorte['chave'] = df_agg_recorte['chave'].str.replace(' 00:00:00', '')

In [47]:
df_agg_vl_bruto_recuperados = df_agg_vl_bruto_recuperados.drop(['dt_mes', 'hierarquia_1', 'hierarquia_4'], axis=1)
df_agg_churn_recuperados = df_agg_churn_recuperados.drop(['dt_mes', 'hierarquia_1', 'hierarquia_4'], axis=1)
df_agg_churn = df_agg_churn.drop(['dt_mes', 'hierarquia_1', 'hierarquia_4'], axis=1)
df_agg_recorte = df_agg_recorte.drop(['dt_mes', 'hierarquia_1', 'hierarquia_4'], axis=1)

In [48]:
df_unificado = pd.merge(df_count_cx, df_agg_vl_bruto_recuperados, on ='chave', how = 'left')
df_unificado = pd.merge(df_unificado, df_agg_recorte, on ='chave', how = 'left')
df_unificado = pd.merge(df_unificado, df_agg_churn, on ='chave', how = 'left')
df_unificado = pd.merge(df_unificado, df_agg_churn_recuperados, on ='chave', how = 'left')

In [49]:
df_unificado = df_unificado.fillna(0.0)

In [50]:
df_unificado

,dt_mes,hierarquia_1,hierarquia_4,clientes_cx,chave,vl_bruto_recuperados_60d,LTV_Receita_Recuperado,LT_mediano_recuperados,clientes_recorte_total,clientes_recorte_churn,clientes_recorte_churn_recuperados
0,2023-06-01,Cartão,Dúvidas sobre pagamento de fatura,2467,2023-06-01CartãoDúvidas sobre pagamento de fatura,153258.556793,440829.413897,24.613308,744.0,174.0,110.0
1,2023-08-01,Cartão,Problemas com a compra,7393,2023-08-01CartãoProblemas com a compra,204125.151426,312677.287884,33.249659,1611.0,122.0,108.0
2,2023-05-01,Tecnologia,Recebimento de PIX,5356,2023-05-01TecnologiaRecebimento de PIX,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,2023-05-01,Conta,Recebimento de PIX,7604,2023-05-01ContaRecebimento de PIX,315169.497588,435271.549913,32.428638,3449.0,321.0,260.0
4,2023-08-01,Crédito,Dúvidas sobre limite de crédito,9678,2023-08-01CréditoDúvidas sobre limite de crédito,191519.350060,718069.295653,32.428638,1654.0,232.0,137.0
...,...,...,...,...,...,...,...,...,...,...,...
3197,2023-07-01,Fraude,Dúvidas FGTS,1,2023-07-01FraudeDúvidas FGTS,0.000000,0.000000,0.000000,0.0,0.0,0.0
3198,2023-06-01,Cobrança,Informações sobre cartão,1,2023-06-01CobrançaInformações sobre cartão,0.000000,0.000000,0.000000,0.0,0.0,0.0
3199,2023-04-01,Fraude,Dúvidas em lançamentos,2,2023-04-01FraudeDúvidas em lançamentos,0.000000,0.000000,0.000000,0.0,0.0,0.0
3200,2023-06-01,Tecnologia,Informações sobre cartão,1,2023-06-01TecnologiaInformações sobre cartão,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [51]:
#df_unificado.to_csv('analise_tabela.csv')

## LTV Geral

In [52]:
query_ltv = """

select
mesref
, sum(ltv) as montante_ltv
, avg(ltv) as media_ltv
, approx_percentile(ltv, 0.5) as mediana_ltv
,  count(distinct l.cpf) as clientes_rentaveis_total
from public.rentabilidade_cartoes_diego_camilo l
left join growth_curated_zone.proposal_analysis pr on (pr.cpf = l.cpf)
where date_diff('month', cast(pr.dt_cfi_account as date), current_date) <= 12
group by 1
order by mesref


"""

In [53]:
ltv_rentaveis = ac.df_athena_q('flavia-costa', query_ltv)

In [54]:
ltv_rentaveis['dt_mes'] = ltv_rentaveis['mesref'] + '-01'
ltv_rentaveis['dt_mes'] = pd.to_datetime(ltv_rentaveis['dt_mes'])

In [55]:
df_unificado['dt_mes'] = pd.to_datetime(df_unificado['dt_mes'])

In [56]:
df_unificado_final = pd.merge(df_unificado, ltv_rentaveis[['dt_mes','montante_ltv','media_ltv','mediana_ltv','clientes_rentaveis_total']], on = 'dt_mes', how = 'left')

In [57]:
df_unificado_final['montante_ltv'] = df_unificado_final['montante_ltv'] * df_unificado_final['LT_mediano_recuperados']
df_unificado_final['media_ltv'] = df_unificado_final['media_ltv'] * df_unificado_final['LT_mediano_recuperados']
df_unificado_final['mediana_ltv'] = df_unificado_final['mediana_ltv'] * df_unificado_final['LT_mediano_recuperados']

In [58]:
df_unificado_final['perc_recorte'] = np.where(df_unificado_final['clientes_cx'] > 0, df_unificado_final['clientes_recorte_total'] / df_unificado_final['clientes_cx'],0)

df_unificado_final['perc_churn_recorte'] = np.where(df_unificado_final['clientes_recorte_total'] > 0, df_unificado_final['clientes_recorte_churn'] / df_unificado_final['clientes_recorte_total'], 0)

df_unificado_final['perc_recuperacao_churn'] = np.where(df_unificado_final['clientes_recorte_churn'] > 0, df_unificado_final['clientes_recorte_churn_recuperados'] / df_unificado_final['clientes_recorte_churn'], 0)

In [59]:
df_unificado_final

,dt_mes,hierarquia_1,hierarquia_4,clientes_cx,chave,vl_bruto_recuperados_60d,LTV_Receita_Recuperado,LT_mediano_recuperados,clientes_recorte_total,clientes_recorte_churn,clientes_recorte_churn_recuperados,montante_ltv,media_ltv,mediana_ltv,clientes_rentaveis_total,perc_recorte,perc_churn_recorte,perc_recuperacao_churn
0,2023-06-01,Cartão,Dúvidas sobre pagamento de fatura,2467,2023-06-01CartãoDúvidas sobre pagamento de fatura,153258.556793,440829.413897,24.613308,744.0,174.0,110.0,2.171191e+08,153.804068,0.002483,1409209,0.301581,0.233871,0.632184
1,2023-08-01,Cartão,Problemas com a compra,7393,2023-08-01CartãoProblemas com a compra,204125.151426,312677.287884,33.249659,1611.0,122.0,108.0,4.227792e+08,221.623745,0.025054,1904578,0.217909,0.075729,0.885246
2,2023-05-01,Tecnologia,Recebimento de PIX,5356,2023-05-01TecnologiaRecebimento de PIX,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,-0.000000,1177656,0.000000,0.000000,0.000000
3,2023-05-01,Conta,Recebimento de PIX,7604,2023-05-01ContaRecebimento de PIX,315169.497588,435271.549913,32.428638,3449.0,321.0,260.0,2.274771e+08,192.814951,-0.002316,1177656,0.453577,0.093070,0.809969
4,2023-08-01,Crédito,Dúvidas sobre limite de crédito,9678,2023-08-01CréditoDúvidas sobre limite de crédito,191519.350060,718069.295653,32.428638,1654.0,232.0,137.0,4.123397e+08,216.151278,0.024435,1904578,0.170903,0.140266,0.590517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3197,2023-07-01,Fraude,Dúvidas FGTS,1,2023-07-01FraudeDúvidas FGTS,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1671878,0.000000,0.000000,0.000000
3198,2023-06-01,Cobrança,Informações sobre cartão,1,2023-06-01CobrançaInformações sobre cartão,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1409209,0.000000,0.000000,0.000000
3199,2023-04-01,Fraude,Dúvidas em lançamentos,2,2023-04-01FraudeDúvidas em lançamentos,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,-0.000000,961931,0.000000,0.000000,0.000000
3200,2023-06-01,Tecnologia,Informações sobre cartão,1,2023-06-01TecnologiaInformações sobre cartão,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1409209,0.000000,0.000000,0.000000


## Exporta para Sandbox

In [60]:
area = 'cx'
name = 'report_perdas_cr.csv'
filename = f'{area}.{name}'

df_unificado_final.drop('chave', axis = 1).to_csv(filename, index=False)

os.system(f'aws s3 cp {filename} s3://data-sandbox-zone-will-prod/upload/')

upload: ./cx.report_perdas_cr.csv to s3://data-sandbox-zone-will-prod/upload/cx.report_perdas_cr.csv


0